In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from patsy import dmatrices

#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#import randomforest classifier
from sklearn.ensemble import RandomForestClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
#import knn classifier
from sklearn.neighbors import KNeighborsClassifier

#for validating your classification model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.grid_search import GridSearchCV

from IPython.display import Image
from IPython.core.display import HTML

C:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [88]:
        #import patient data
        df = pd.read_csv('data/Challenge_1_Training_partially_clean.csv')
        df.head(5)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,22915332,1475073,Caucasian,Female,[80-90),?,3,1,4,5,...,0,0,0,0,0,0,0,No,Yes,>30
1,158361324,93771396,Caucasian,Female,[70-80),?,5,3,1,6,...,0,3,0,0,0,0,0,Ch,Yes,NO
2,120453192,24581277,Other,Female,[60-70),?,1,22,7,4,...,0,2,0,0,0,0,0,No,Yes,NO
3,25590894,5041395,Caucasian,Male,[70-80),?,1,1,7,3,...,0,2,0,0,0,0,0,No,Yes,>30
4,154290822,49027563,Caucasian,Female,[30-40),?,2,1,1,3,...,0,0,0,0,0,0,0,No,No,NO


In [89]:
#show the information about the data')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 50 columns):
encounter_id                56000 non-null int64
patient_nbr                 56000 non-null int64
race                        56000 non-null object
gender                      56000 non-null object
age                         56000 non-null object
weight                      56000 non-null object
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null object
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG

In [90]:
#describe the column readmitted only (e.g., count, unique, frequency)
df['readmitted'].describe()

count     56000
unique        3
top          NO
freq      30238
Name: readmitted, dtype: object

In [91]:
#distribution of 0 and 1 in the readmitted column
df.groupby('readmitted').count()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed
readmitted,,,,,,,,,,,,,,,,,,,,,
<30,6285,6285,6285,6285,6285,6285,6285,6285,6285,6285,...,6285,6285,6285,6285,6285,6285,6285,6285,6285,6285
>30,19477,19477,19477,19477,19477,19477,19477,19477,19477,19477,...,19477,19477,19477,19477,19477,19477,19477,19477,19477,19477
NO,30238,30238,30238,30238,30238,30238,30238,30238,30238,30238,...,30238,30238,30238,30238,30238,30238,30238,30238,30238,30238


In [92]:
#replace the values of the 'readmitted' column:
# NO = 0
# >30 = 1
# <30 = 2

df = df.replace({'readmitted': {'NO': 0, '>30': 1, '<30': 2}})

df.head(2)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,22915332,1475073,Caucasian,Female,[80-90),?,3,1,4,5,...,0,0,0,0,0,0,0,No,Yes,1
1,158361324,93771396,Caucasian,Female,[70-80),?,5,3,1,6,...,0,3,0,0,0,0,0,Ch,Yes,0


# Data understanding & processing (ETL)

In [93]:
#drop or remove the columns 'encounter_id' and 'patient_nbr' since this column is not used in the analysis and disply the result
df = df.drop('encounter_id', axis=1)
df = df.drop('patient_nbr', axis=1)
df.head(5)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[80-90),?,3,1,4,5,?,0,...,0,0,0,0,0,0,0,No,Yes,1
1,Caucasian,Female,[70-80),?,5,3,1,6,MC,0,...,0,3,0,0,0,0,0,Ch,Yes,0
2,Other,Female,[60-70),?,1,22,7,4,SP,18,...,0,2,0,0,0,0,0,No,Yes,0
3,Caucasian,Male,[70-80),?,1,1,7,3,?,18,...,0,2,0,0,0,0,0,No,Yes,1
4,Caucasian,Female,[30-40),?,2,1,1,3,?,0,...,0,0,0,0,0,0,0,No,No,0


In [94]:
df.groupby('race').count()

,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
race,,,,,,,,,,,,,,,,,,,,,
?,1215,1215,1215,1215,1215,1215,1215,1215,1215,1215,...,1215,1215,1215,1215,1215,1215,1215,1215,1215,1215
AfricanAmerican,10563,10563,10563,10563,10563,10563,10563,10563,10563,10563,...,10563,10563,10563,10563,10563,10563,10563,10563,10563,10563
Asian,356,356,356,356,356,356,356,356,356,356,...,356,356,356,356,356,356,356,356,356,356
Caucasian,41886,41886,41886,41886,41886,41886,41886,41886,41886,41886,...,41886,41886,41886,41886,41886,41886,41886,41886,41886,41886
Hispanic,1117,1117,1117,1117,1117,1117,1117,1117,1117,1117,...,1117,1117,1117,1117,1117,1117,1117,1117,1117,1117
Other,863,863,863,863,863,863,863,863,863,863,...,863,863,863,863,863,863,863,863,863,863


In [95]:
#replace the values of the 'race' column:
# ? = 0
# AfricanAmerican = 1
# Asian = 2
# Caucasion = 3
# Hispanic = 4
# Other = 5

df = df.replace({'race': {'?': 0, 'AfricanAmerican': 1, 'Asian': 2,'Caucasian': 3,'Hispanic': 4,'Other': 5}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,Female,[80-90),?,3,1,4,5,?,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,Female,[70-80),?,5,3,1,6,MC,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [96]:
df.groupby('gender').count()

,race,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
gender,,,,,,,,,,,,,,,,,,,,,
Female,29990,29990,29990,29990,29990,29990,29990,29990,29990,29990,...,29990,29990,29990,29990,29990,29990,29990,29990,29990,29990
Male,26010,26010,26010,26010,26010,26010,26010,26010,26010,26010,...,26010,26010,26010,26010,26010,26010,26010,26010,26010,26010


In [97]:
#replace the values of the 'gender' column:
# Female = 0
# Male = 1

df = df.replace({'gender': {'Male': 1, 'Female': 0}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,[80-90),?,3,1,4,5,?,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,[70-80),?,5,3,1,6,MC,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [98]:
df.groupby('age').count()

,race,gender,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
age,,,,,,,,,,,,,,,,,,,,,
[0-10),98,98,98,98,98,98,98,98,98,98,...,98,98,98,98,98,98,98,98,98,98
[10-20),355,355,355,355,355,355,355,355,355,355,...,355,355,355,355,355,355,355,355,355,355
[20-30),934,934,934,934,934,934,934,934,934,934,...,934,934,934,934,934,934,934,934,934,934
[30-40),2070,2070,2070,2070,2070,2070,2070,2070,2070,2070,...,2070,2070,2070,2070,2070,2070,2070,2070,2070,2070
[40-50),5237,5237,5237,5237,5237,5237,5237,5237,5237,5237,...,5237,5237,5237,5237,5237,5237,5237,5237,5237,5237
[50-60),9578,9578,9578,9578,9578,9578,9578,9578,9578,9578,...,9578,9578,9578,9578,9578,9578,9578,9578,9578,9578
[60-70),12422,12422,12422,12422,12422,12422,12422,12422,12422,12422,...,12422,12422,12422,12422,12422,12422,12422,12422,12422,12422
[70-80),14356,14356,14356,14356,14356,14356,14356,14356,14356,14356,...,14356,14356,14356,14356,14356,14356,14356,14356,14356,14356
[80-90),9436,9436,9436,9436,9436,9436,9436,9436,9436,9436,...,9436,9436,9436,9436,9436,9436,9436,9436,9436,9436


In [99]:
#replace the values of the 'age' column:
# [0-10) = 0
# [10-20) = 1
# [20-30) = 2
# [30-40) = 3
# [40-50) = 4
# [50-60) = 5
# [60-70) = 6
# [70-80) = 7
# [80-90) = 8
# [90-100) = 9

df = df.replace({'age': {'[0-10)': 0, '[10-20)': 1, '[20-30)': 2, '[30-40)': 3, '[40-50)': 4, '[50-60)': 5, '[60-70)': 6, '[70-80)': 7, '[80-90)': 8, '[90-100)': 9}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,?,3,1,4,5,?,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,7,?,5,3,1,6,MC,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [100]:
df.groupby('weight').count()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
weight,,,,,,,,,,,,,,,,,,,,,
>200,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
?,54238,54238,54238,54238,54238,54238,54238,54238,54238,54238,...,54238,54238,54238,54238,54238,54238,54238,54238,54238,54238
[0-25),27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
[100-125),349,349,349,349,349,349,349,349,349,349,...,349,349,349,349,349,349,349,349,349,349
[125-150),67,67,67,67,67,67,67,67,67,67,...,67,67,67,67,67,67,67,67,67,67
[150-175),19,19,19,19,19,19,19,19,19,19,...,19,19,19,19,19,19,19,19,19,19
[175-200),7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
[25-50),58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58
[50-75),488,488,488,488,488,488,488,488,488,488,...,488,488,488,488,488,488,488,488,488,488


In [101]:
#replace the values of the 'weight' column:
# ? = 0
# [0-25) = 1
# [25-50) = 2
# [50-75) = 3
# [75-100) = 4
# [100-125) = 5
# [125-150) = 6
# [150-175) = 7
# [175-200) = 8
# > 200 = 9

df = df.replace({'weight': {'?': 0, '[0-25)': 1, '[25-50)': 2, '[50-75)': 3, '[75-100)': 4, '[100-125)': 5, '[125-150)': 6, '[150-175)': 7, '[175-200)': 8, '>200': 9}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,?,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,7,0,5,3,1,6,MC,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [102]:
df.groupby('payer_code').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,MED_SPEC_NUM,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
payer_code,,,,,,,,,,,,,,,,,,,,,
?,22153,22153,22153,22153,22153,22153,22153,22153,22153,22153,...,22153,22153,22153,22153,22153,22153,22153,22153,22153,22153
BC,2550,2550,2550,2550,2550,2550,2550,2550,2550,2550,...,2550,2550,2550,2550,2550,2550,2550,2550,2550,2550
CH,81,81,81,81,81,81,81,81,81,81,...,81,81,81,81,81,81,81,81,81,81
CM,1044,1044,1044,1044,1044,1044,1044,1044,1044,1044,...,1044,1044,1044,1044,1044,1044,1044,1044,1044,1044
CP,1405,1405,1405,1405,1405,1405,1405,1405,1405,1405,...,1405,1405,1405,1405,1405,1405,1405,1405,1405,1405
DM,309,309,309,309,309,309,309,309,309,309,...,309,309,309,309,309,309,309,309,309,309
HM,3506,3506,3506,3506,3506,3506,3506,3506,3506,3506,...,3506,3506,3506,3506,3506,3506,3506,3506,3506,3506
MC,17855,17855,17855,17855,17855,17855,17855,17855,17855,17855,...,17855,17855,17855,17855,17855,17855,17855,17855,17855,17855
MD,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977,...,1977,1977,1977,1977,1977,1977,1977,1977,1977,1977


In [103]:
#replace the values of the 'payer_code' column:

df = df.replace({'payer_code': {'?': 0, 'BC': 1, 'CH': 2, 'CM': 3, 'CP': 4, 'DM': 5, 'HM': 6, 'MC': 7, 'MD': 8, 'MP': 9, 'OG': 10, 'OT': 11, 'PO': 12, 'SI': 13, 'SP': 14, 'UN': 15, 'WC': 16}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,0,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,7,0,5,3,1,6,7,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [106]:
df.groupby('MED_SPEC_NUM').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,num_lab_procedures,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
MED_SPEC_NUM,,,,,,,,,,,,,,,,,,,,,
0,27562,27562,27562,27562,27562,27562,27562,27562,27562,27562,...,27562,27562,27562,27562,27562,27562,27562,27562,27562,27562
1,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
3,12,12,12,12,12,12,12,12,12,12,...,12,12,12,12,12,12,12,12,12,12
4,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912,...,2912,2912,2912,2912,2912,2912,2912,2912,2912,2912
5,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
6,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
8,4189,4189,4189,4189,4189,4189,4189,4189,4189,4189,...,4189,4189,4189,4189,4189,4189,4189,4189,4189,4189


In [108]:
df.groupby('max_glu_serum').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
max_glu_serum,,,,,,,,,,,,,,,,,,,,,
>200,800,800,800,800,800,800,800,800,800,800,...,800,800,800,800,800,800,800,800,800,800
>300,692,692,692,692,692,692,692,692,692,692,...,692,692,692,692,692,692,692,692,692,692
None,53027,53027,53027,53027,53027,53027,53027,53027,53027,53027,...,53027,53027,53027,53027,53027,53027,53027,53027,53027,53027
Norm,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481,...,1481,1481,1481,1481,1481,1481,1481,1481,1481,1481


In [109]:
#replace the values of the 'max_glu_serum' column:
# None = 0
# Norm = 1
# >200 = 2
# >300 = 3

df = df.replace({'max_glu_serum': {'None': 0, 'Norm': 1, '>200': 2, '>300': 3}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,0,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,7,0,5,3,1,6,7,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [111]:
df.groupby('A1Cresult').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
A1Cresult,,,,,,,,,,,,,,,,,,,,,
>7,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,...,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143
>8,4523,4523,4523,4523,4523,4523,4523,4523,4523,4523,...,4523,4523,4523,4523,4523,4523,4523,4523,4523,4523
None,46560,46560,46560,46560,46560,46560,46560,46560,46560,46560,...,46560,46560,46560,46560,46560,46560,46560,46560,46560,46560
Norm,2774,2774,2774,2774,2774,2774,2774,2774,2774,2774,...,2774,2774,2774,2774,2774,2774,2774,2774,2774,2774


In [113]:
#replace the values of the 'A1Cresult' column:
# None = 0
# Norm = 1
# >7 = 2
# >8 = 3

df = df.replace({'A1Cresult': {'None': 0, 'Norm': 1, '>7': 2, '>8': 3}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,0,0,...,0,0,0,0,0,0,0,No,Yes,1
1,3,0,7,0,5,3,1,6,7,0,...,0,3,0,0,0,0,0,Ch,Yes,0


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [115]:
df.groupby('change').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,diabetesMed,readmitted
change,,,,,,,,,,,,,,,,,,,,,
Ch,25910,25910,25910,25910,25910,25910,25910,25910,25910,25910,...,25910,25910,25910,25910,25910,25910,25910,25910,25910,25910
No,30090,30090,30090,30090,30090,30090,30090,30090,30090,30090,...,30090,30090,30090,30090,30090,30090,30090,30090,30090,30090


In [116]:
#replace the values of the 'change' column:
# No = 0
# Ch = 1

df = df.replace({'change': {'No': 0, 'Ch': 1}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,0,0,...,0,0,0,0,0,0,0,0,Yes,1
1,3,0,7,0,5,3,1,6,7,0,...,0,3,0,0,0,0,0,1,Yes,0


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [118]:
df.groupby('diabetesMed').count()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,examide,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,readmitted
diabetesMed,,,,,,,,,,,,,,,,,,,,,
No,12890,12890,12890,12890,12890,12890,12890,12890,12890,12890,...,12890,12890,12890,12890,12890,12890,12890,12890,12890,12890
Yes,43110,43110,43110,43110,43110,43110,43110,43110,43110,43110,...,43110,43110,43110,43110,43110,43110,43110,43110,43110,43110


In [121]:
#replace the values of the 'diabetesMed' column:
# No = 0
# Yes = 1


df = df.replace({'diabetesMed': {'No': 0, 'Yes': 1,}})

df.head(2)

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,MED_SPEC_NUM,...,citoglipton,insulin,glyburide.metformin,glipizide.metformin,glimepiride.pioglitazone,metformin.rosiglitazone,metformin.pioglitazone,change,diabetesMed,readmitted
0,3,0,8,0,3,1,4,5,0,0,...,0,0,0,0,0,0,0,0,1,1
1,3,0,7,0,5,3,1,6,7,0,...,0,3,0,0,0,0,0,1,1,0


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [123]:
df_clean_NoString = df

In [124]:
df_clean_NoString.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 48 columns):
race                        56000 non-null int64
gender                      56000 non-null int64
age                         56000 non-null int64
weight                      56000 non-null int64
admission_type_id           56000 non-null int64
discharge_disposition_id    56000 non-null int64
admission_source_id         56000 non-null int64
time_in_hospital            56000 non-null int64
payer_code                  56000 non-null int64
MED_SPEC_NUM                56000 non-null int64
num_lab_procedures          56000 non-null int64
num_procedures              56000 non-null int64
num_medications             56000 non-null int64
number_outpatient           56000 non-null int64
number_emergency            56000 non-null int64
number_inpatient            56000 non-null int64
DIAG_1_NUM                  56000 non-null int64
DIAG_2_NUM                  56000 non-null int64
DIAG_3_NU

In [125]:
df_clean_NoString.to_csv('data/df_clean_NoString.csv')